# Extract Foercasted Weather Data
# -------------------------------

## Initial Imports

In [1]:
# Import Dependencies
import requests
import pandas as pd
import json
import numpy as np
import datetime
from dateutil import tz

In [2]:
# Import API Key
from config import weather_api_key

In [3]:
# Documentation
# https://openweathermap.org/api/one-call-api

# Define the Functions
# --------------------

In [4]:
def convert_DateTime_UTC_to_CST(UTC_datetime_list, list_range):
    CST_datetime_list = []

    for date in list_range:    
        # Convert the date/time to ISO standard in string format
        date_time = datetime.datetime.utcfromtimestamp(UTC_datetime_list[date]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Create a datetime object, representing the UTC time
        time_utc = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S')

        # Replace the timezone field of the datetime object to UTC
        from_zone = tz.gettz('UTC')
        time_utc = time_utc.replace(tzinfo=from_zone)

        # Convert time zone from UTC to central
        to_zone = tz.gettz('America/Chicago')
        time_cst = time_utc.astimezone(to_zone)

        # Append the date/time, year, month, day, and hour
        CST_datetime_list.append({
            "UTC_Time": UTC_datetime_list[date],
            "Date_Time": time_cst.strftime('%Y-%m-%d %H:%M:%S'),
            "Year": time_cst.year,
            "Month":time_cst.month,
            "Day":time_cst.day,
            "Hour":time_cst.hour
            })

    datetimeDataFrame = pd.DataFrame(CST_datetime_list)
    
    return datetimeDataFrame

In [5]:
def calculate_sunhour(sunrise_list, sunset_list, list_range):
    
    sunhour_list = []

    for day in list_range:
        # Convert the date/time to ISO standard in string format
        sunrise_date_time = datetime.datetime.utcfromtimestamp(sunrise_list[day]).strftime('%Y-%m-%d %H:%M:%S')
        sunset_date_time = datetime.datetime.utcfromtimestamp(sunset_list[day]).strftime('%Y-%m-%d %H:%M:%S')

        # Create a datetime object, representing the UTC time
        sunrise_utc = datetime.datetime.strptime(sunrise_date_time, '%Y-%m-%d %H:%M:%S')
        sunset_utc = datetime.datetime.strptime(sunset_date_time, '%Y-%m-%d %H:%M:%S')

        # Replace the timezone field of the datetime object to UTC
        from_zone = tz.gettz('UTC')
        
        sunrise_utc = sunrise_utc.replace(tzinfo=from_zone)
        sunset_utc = sunset_utc.replace(tzinfo=from_zone)

        # Convert time zone from UTC to central
        to_zone = tz.gettz('America/Chicago')
        
        sunrise_cst = sunrise_utc.astimezone(to_zone)
        sunset_cst = sunset_utc.astimezone(to_zone)
        
        # Convert to string
        sunrise_str = sunrise_cst.strftime('%Y-%m-%d %H:%M:%S')
        sunset_str = sunset_cst.strftime('%Y-%m-%d %H:%M:%S')

        # Calculate Sunhour
        sunrise = datetime.datetime.strptime(sunrise_str, '%Y-%m-%d %H:%M:%S')
        sunset = datetime.datetime.strptime(sunset_str, '%Y-%m-%d %H:%M:%S')
        Sunhour_timedelta = sunset - sunrise
        Sunhour_seconds = Sunhour_timedelta.seconds
        Sunhour = Sunhour_seconds / 3600

        # Append to List
        sunhour_list.append({
            "Sunrise": sunrise_list[day],
            "Sunhour": Sunhour
        })
        
    sunhourDataFrame = pd.DataFrame(sunhour_list)
    
    return sunhourDataFrame

# Webberville Solar Farm
# ----------------------

In [6]:
# Geographical coordinates (latitude, longitude)
lat = "30.238333"
lon = "-97.508611"

# Exclude some parts of the weather data from the API response
    # It should be a comma-delimited list (without spaces)
    # Options: current,minutely,hourly,daily,alerts
part = "minutely,alerts"

# Define the Units of measurement
    # Options: stadnard,metric,imperial
units = "imperial"

# Make a request to openweathermap
requestURL = f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&units={units}&appid={weather_api_key}"

# print(requestURL)

In [7]:
response = requests.get(requestURL)

if response.status_code == 200:
    # Turn the response into a JSON object
    responseJson = response.json()
    print("Successfully turned response into JSON object.")
else:
    # Else, print the Error status code
    print("The Error Status Code is:") 
    print(response.status_code)

Successfully turned response into JSON object.


# Current Weather Data:

## Date / Time

In [8]:
# Convert the date/time to ISO standard in string format
current_date_time = datetime.datetime.utcfromtimestamp(responseJson["current"]["dt"]).strftime('%Y-%m-%d %H:%M:%S')

# Create a datetime object, representing the UTC time
current_time_utc = datetime.datetime.strptime(current_date_time, '%Y-%m-%d %H:%M:%S')

# Replace the timezone field of the datetime object to UTC
from_zone = tz.gettz('UTC')
current_time_utc = current_time_utc.replace(tzinfo=from_zone)

# Convert time zone from UTC to central
to_zone = tz.gettz('America/Chicago')
current_time_cst = current_time_utc.astimezone(to_zone)

# Display the current time
current_time_cst

datetime.datetime(2020, 11, 1, 14, 52, 59, tzinfo=tzfile('US/Central'))

In [9]:
# Extract the date & time and store as a string
current_Date_Time = current_time_cst.strftime('%Y-%m-%d %H:%M:%S')

# Display the date & time as string
current_Date_Time

'2020-11-01 14:52:59'

In [10]:
# Extract the date and store as a string
current_Date = datetime.datetime.date(current_time_cst)
current_Date = current_Date.strftime('%Y-%m-%d')

# Display the date in %Y-%m-%d
current_Date

'2020-11-01'

In [11]:
# Exract the time in %H:%M:%S
current_Time = current_time_cst.strftime('%H:%M:%S')

# Display the hour
current_Time

'14:52:59'

In [12]:
# Extract the year from the current time
current_Year = current_time_cst.year

# Display the year
current_Year

2020

In [13]:
# Extract the month from the current time
current_Month = current_time_cst.month

# Display the year
current_Month

11

In [14]:
# Extract the day from the current time
current_Day = current_time_cst.day

# Display the day
current_Day

1

In [15]:
# Extract the hour from the current time
current_Hour = current_time_cst.hour

# Display the day
current_Hour

14

In [16]:
# Alternative:
# Extract the hour from the current time
current_Hour = datetime.datetime.time(current_time_cst).hour

# Display the hour
current_Hour

14

In [17]:
# Store the next day as a variable
next_day = datetime.date(current_Year, current_Month, current_Day) + datetime.timedelta(days=1)
next_day

datetime.date(2020, 11, 2)

## Weather Description

In [18]:
# Extract the current weather description
current_Weather_Description = responseJson["current"]["weather"][0]["description"]

# Display the weather description
current_Weather_Description

'broken clouds'

## Temperature

In [19]:
# Extract the current temperature (F)
current_Temperature_F = responseJson["current"]["temp"]

# Display the temperature
current_Temperature_F

76.75

## Sunhour

In [20]:
# Sunrise:

# Convert the date/time to ISO standard in string format
sunrise_date_time = datetime.datetime.utcfromtimestamp(responseJson["current"]["sunrise"]).strftime('%Y-%m-%d %H:%M:%S')

# Create a datetime object, representing the UTC time
sunrise_utc = datetime.datetime.strptime(sunrise_date_time, '%Y-%m-%d %H:%M:%S')

# Replace the timezone field of the datetime object to UTC
from_zone = tz.gettz('UTC')
sunrise_utc = sunrise_utc.replace(tzinfo=from_zone)

# Convert time zone from UTC to central
to_zone = tz.gettz('America/Chicago')
todays_sunrise = sunrise_utc.astimezone(to_zone)

# Convert to string
todays_sunrise = todays_sunrise.strftime('%Y-%m-%d %H:%M:%S')

# Display sunrise time
todays_sunrise

'2020-11-01 06:44:37'

In [21]:
# Sunset:

# Convert the date/time to ISO standard in string format
sunset_date_time = datetime.datetime.utcfromtimestamp(responseJson["current"]["sunset"]).strftime('%Y-%m-%d %H:%M:%S')

# Create a datetime object, representing the UTC time
sunset_utc = datetime.datetime.strptime(sunset_date_time, '%Y-%m-%d %H:%M:%S')

# Replace the timezone field of the datetime object to UTC
from_zone = tz.gettz('UTC')
sunset_utc = sunset_utc.replace(tzinfo=from_zone)

# Convert time zone from UTC to central
to_zone = tz.gettz('America/Chicago')
todays_sunset = sunset_utc.astimezone(to_zone)

# Convert to string
todays_sunset = todays_sunset.strftime('%Y-%m-%d %H:%M:%S')

# Display sunrise time
todays_sunset

'2020-11-01 17:42:29'

In [22]:
todays_sunrise = datetime.datetime.strptime(todays_sunrise, '%Y-%m-%d %H:%M:%S')
todays_sunset = datetime.datetime.strptime(todays_sunset, '%Y-%m-%d %H:%M:%S')
Sunhour_timedelta = todays_sunset - todays_sunrise
print(f"The sunhour time in %H:%M:%S : {Sunhour_timedelta}")
Sunhour_seconds = Sunhour_timedelta.seconds
Sunhour = Sunhour_seconds / 3600
print(f"The sunhour time in hrs : {Sunhour}")

The sunhour time in %H:%M:%S : 10:57:52
The sunhour time in hrs : 10.964444444444444


## Cloud Cover

In [23]:
# Extract the current cloud cover (%)
current_CloudCover_percent = responseJson["current"]["clouds"]

# Display the Cloudiness, %
current_CloudCover_percent

75

## uv Index

In [24]:
# Extract the current UV Index
todays_uvIndex = responseJson["current"]["uvi"]

# Display the Midday UV index
todays_uvIndex

5.49

## Humidity

In [25]:
# Extract the current humidity (%)
current_Humidity_percent = responseJson["current"]["humidity"]

# Display the Cloudiness, %
current_Humidity_percent

34

# Daily Forecasted Weather Data:

## Create a dataframe for the daily forecasted weather data

In [26]:
forecasted_daily_weather = []

for day in np.arange(0, 8, 1):
    forecasted_daily_weather.append({
        "UTC_Time": responseJson["daily"][day]["dt"],
        "Sunrise": responseJson["daily"][day]["sunrise"],
        "Sunset": responseJson["daily"][day]["sunset"],
        "uvIndex": responseJson["daily"][day]["uvi"]
    })

daily_weather_DF = pd.DataFrame(forecasted_daily_weather)
daily_weather_DF

,UTC_Time,Sunrise,Sunset,uvIndex
0,1604253600,1604234677,1604274149,5.49
1,1604340000,1604321125,1604360502,5.12
2,1604426400,1604407573,1604446856,5.19
3,1604512800,1604494021,1604533211,5.05
4,1604599200,1604580469,1604619568,4.74
5,1604685600,1604666918,1604705926,4.90
6,1604772000,1604753368,1604792286,4.97
7,1604858400,1604839817,1604878647,5.20


## Clean up the UTC_Time column: Convert UTC time to CST

In [27]:
daily_index = np.arange(0,8,1)

daily_UTC_date_time = daily_weather_DF["UTC_Time"]

date_time_DF = convert_DateTime_UTC_to_CST(daily_UTC_date_time, daily_index)

date_time_DF

,UTC_Time,Date_Time,Year,Month,Day,Hour
0,1604253600,2020-11-01 12:00:00,2020,11,1,12
1,1604340000,2020-11-02 12:00:00,2020,11,2,12
2,1604426400,2020-11-03 12:00:00,2020,11,3,12
3,1604512800,2020-11-04 12:00:00,2020,11,4,12
4,1604599200,2020-11-05 12:00:00,2020,11,5,12
5,1604685600,2020-11-06 12:00:00,2020,11,6,12
6,1604772000,2020-11-07 12:00:00,2020,11,7,12
7,1604858400,2020-11-08 12:00:00,2020,11,8,12


In [28]:
daily_weather_DF = pd.merge(daily_weather_DF, date_time_DF, on='UTC_Time', how='outer')
daily_weather_DF.drop(columns=["UTC_Time"], axis=1, inplace=True)
daily_weather_DF["Date_Time"] = pd.to_datetime(daily_weather_DF["Date_Time"])
daily_weather_DF

,Sunrise,Sunset,uvIndex,Date_Time,Year,Month,Day,Hour
0,1604234677,1604274149,5.49,2020-11-01 12:00:00,2020,11,1,12
1,1604321125,1604360502,5.12,2020-11-02 12:00:00,2020,11,2,12
2,1604407573,1604446856,5.19,2020-11-03 12:00:00,2020,11,3,12
3,1604494021,1604533211,5.05,2020-11-04 12:00:00,2020,11,4,12
4,1604580469,1604619568,4.74,2020-11-05 12:00:00,2020,11,5,12
5,1604666918,1604705926,4.90,2020-11-06 12:00:00,2020,11,6,12
6,1604753368,1604792286,4.97,2020-11-07 12:00:00,2020,11,7,12
7,1604839817,1604878647,5.20,2020-11-08 12:00:00,2020,11,8,12


## Use sunrise and sunset times to calculate the total hours of sunhour for each day

In [29]:
daily_index = np.arange(0,8,1)

UTC_sunrise = daily_weather_DF["Sunrise"]
UTC_sunset = daily_weather_DF["Sunset"]

sunhour_DF = calculate_sunhour(UTC_sunrise, UTC_sunset, daily_index)

sunhour_DF

,Sunrise,Sunhour
0,1604234677,10.964444
1,1604321125,10.938056
2,1604407573,10.911944
3,1604494021,10.886111
4,1604580469,10.860833
5,1604666918,10.835556
6,1604753368,10.810556
7,1604839817,10.786111


In [30]:
daily_weather_DF = pd.merge(daily_weather_DF, sunhour_DF, on='Sunrise', how='outer')
daily_weather_DF.drop(columns=["Sunrise", "Sunset"], axis=1, inplace=True)
daily_weather_DF

,uvIndex,Date_Time,Year,Month,Day,Hour,Sunhour
0,5.49,2020-11-01 12:00:00,2020,11,1,12,10.964444
1,5.12,2020-11-02 12:00:00,2020,11,2,12,10.938056
2,5.19,2020-11-03 12:00:00,2020,11,3,12,10.911944
3,5.05,2020-11-04 12:00:00,2020,11,4,12,10.886111
4,4.74,2020-11-05 12:00:00,2020,11,5,12,10.860833
5,4.90,2020-11-06 12:00:00,2020,11,6,12,10.835556
6,4.97,2020-11-07 12:00:00,2020,11,7,12,10.810556
7,5.20,2020-11-08 12:00:00,2020,11,8,12,10.786111


## Display the final daily forecasted weather dataframe

In [31]:
# Re-Order the columns
daily_weather_DF = daily_weather_DF[["Date_Time", "Year", "Month", "Day", "Hour", "Sunhour","uvIndex"]]
daily_weather_DF

,Date_Time,Year,Month,Day,Hour,Sunhour,uvIndex
0,2020-11-01 12:00:00,2020,11,1,12,10.964444,5.49
1,2020-11-02 12:00:00,2020,11,2,12,10.938056,5.12
2,2020-11-03 12:00:00,2020,11,3,12,10.911944,5.19
3,2020-11-04 12:00:00,2020,11,4,12,10.886111,5.05
4,2020-11-05 12:00:00,2020,11,5,12,10.860833,4.74
5,2020-11-06 12:00:00,2020,11,6,12,10.835556,4.90
6,2020-11-07 12:00:00,2020,11,7,12,10.810556,4.97
7,2020-11-08 12:00:00,2020,11,8,12,10.786111,5.20


In [32]:
# Check the data types
daily_weather_DF.dtypes

Date_Time    datetime64[ns]
Year                  int64
Month                 int64
Day                   int64
Hour                  int64
Sunhour             float64
uvIndex             float64
dtype: object

## Clean the dataframe so it can be merged with the hourly forecasted data

In [33]:
daily_weather_DF.drop(columns=["Date_Time", "Year", "Month", "Hour"], axis=1, inplace=True)
daily_weather_DF

,Day,Sunhour,uvIndex
0,1,10.964444,5.49
1,2,10.938056,5.12
2,3,10.911944,5.19
3,4,10.886111,5.05
4,5,10.860833,4.74
5,6,10.835556,4.90
6,7,10.810556,4.97
7,8,10.786111,5.20


# Hourly Forecasted Weather Data:

In [34]:
forecasted_hourly_weather = []

for hour in np.arange(0, 48, 1):
    forecasted_hourly_weather.append({
        "UTC_Time": responseJson["hourly"][hour]["dt"],
        "Temperature_F": responseJson["hourly"][hour]["temp"],
        "Weather_Description": responseJson["hourly"][hour]["weather"][0]["description"],
        "CloudCover_percent": responseJson["hourly"][hour]["clouds"],
        "Humidity_percent": responseJson["hourly"][hour]["humidity"]
    })

hourly_weather_DF = pd.DataFrame(forecasted_hourly_weather)
hourly_weather_DF.head()

,UTC_Time,Temperature_F,Weather_Description,CloudCover_percent,Humidity_percent
0,1604260800,76.75,broken clouds,75,34
1,1604264400,75.54,overcast clouds,87,37
2,1604268000,73.99,overcast clouds,95,40
3,1604271600,70.83,overcast clouds,96,43
4,1604275200,65.34,overcast clouds,96,49


In [35]:
hourly_index = np.arange(0,48,1)

hourly_UTC_date_time = hourly_weather_DF["UTC_Time"]

hourly_date_time_DF = convert_DateTime_UTC_to_CST(hourly_UTC_date_time, hourly_index)

hourly_date_time_DF.head()

,UTC_Time,Date_Time,Year,Month,Day,Hour
0,1604260800,2020-11-01 14:00:00,2020,11,1,14
1,1604264400,2020-11-01 15:00:00,2020,11,1,15
2,1604268000,2020-11-01 16:00:00,2020,11,1,16
3,1604271600,2020-11-01 17:00:00,2020,11,1,17
4,1604275200,2020-11-01 18:00:00,2020,11,1,18


In [36]:
hourly_weather_DF = pd.merge(hourly_weather_DF, hourly_date_time_DF, on='UTC_Time', how='outer')
hourly_weather_DF.drop(columns=["UTC_Time"], axis=1, inplace=True)
hourly_weather_DF["Date_Time"] = pd.to_datetime(hourly_weather_DF["Date_Time"])
hourly_weather_DF = hourly_weather_DF[["Date_Time", "Year", "Month", "Day", "Hour", "Temperature_F", "Humidity_percent", "CloudCover_percent", "Weather_Description"]]
hourly_weather_DF.head()

,Date_Time,Year,Month,Day,Hour,Temperature_F,Humidity_percent,CloudCover_percent,Weather_Description
0,2020-11-01 14:00:00,2020,11,1,14,76.75,34,75,broken clouds
1,2020-11-01 15:00:00,2020,11,1,15,75.54,37,87,overcast clouds
2,2020-11-01 16:00:00,2020,11,1,16,73.99,40,95,overcast clouds
3,2020-11-01 17:00:00,2020,11,1,17,70.83,43,96,overcast clouds
4,2020-11-01 18:00:00,2020,11,1,18,65.34,49,96,overcast clouds


In [37]:
# Check the data types
hourly_weather_DF.dtypes

Date_Time              datetime64[ns]
Year                            int64
Month                           int64
Day                             int64
Hour                            int64
Temperature_F                 float64
Humidity_percent                int64
CloudCover_percent              int64
Weather_Description            object
dtype: object

# Merge the Daily and Hourly Forecasted Weather DataFrames:

In [38]:
forecasted_weather_DF = pd.merge(hourly_weather_DF, daily_weather_DF, on='Day', how='inner')
forecasted_weather_DF = forecasted_weather_DF[["Date_Time", "Year", "Month", "Day", "Hour", "Temperature_F", "Humidity_percent", "Sunhour", "CloudCover_percent", "uvIndex", "Weather_Description"]]

forecasted_weather_DF

,Date_Time,Year,Month,Day,Hour,Temperature_F,Humidity_percent,Sunhour,CloudCover_percent,uvIndex,Weather_Description
0,2020-11-01 14:00:00,2020,11,1,14,76.75,34,10.964444,75,5.49,broken clouds
1,2020-11-01 15:00:00,2020,11,1,15,75.54,37,10.964444,87,5.49,overcast clouds
2,2020-11-01 16:00:00,2020,11,1,16,73.99,40,10.964444,95,5.49,overcast clouds
3,2020-11-01 17:00:00,2020,11,1,17,70.83,43,10.964444,96,5.49,overcast clouds
4,2020-11-01 18:00:00,2020,11,1,18,65.34,49,10.964444,96,5.49,overcast clouds
5,2020-11-01 19:00:00,2020,11,1,19,61.93,52,10.964444,78,5.49,broken clouds
6,2020-11-01 20:00:00,2020,11,1,20,60.40,53,10.964444,90,5.49,overcast clouds
7,2020-11-01 21:00:00,2020,11,1,21,59.47,49,10.964444,93,5.49,overcast clouds
8,2020-11-01 22:00:00,2020,11,1,22,57.94,48,10.964444,94,5.49,overcast clouds
9,2020-11-01 23:00:00,2020,11,1,23,56.12,49,10.964444,95,5.49,overcast clouds


In [39]:
forecasted_weather_DF.dtypes

Date_Time              datetime64[ns]
Year                            int64
Month                           int64
Day                             int64
Hour                            int64
Temperature_F                 float64
Humidity_percent                int64
Sunhour                       float64
CloudCover_percent              int64
uvIndex                       float64
Weather_Description            object
dtype: object

# Hackberry Wind Farm
# -------------------

# Weather Descriptions
# --------------------

In [40]:
weather_description_list = [
    # Clear
    'Clear',
    'Sunny',
    # Clouds
    'Partly cloudy', 
    'Cloudy', 
    'Overcast',
    # Drizzle
    'Freezing drizzle',
    'Light drizzle', 
    'Patchy light drizzle',
    # Snow
    'Patchy moderate snow', 
    'Moderate snow', 
    'Light sleet',
    'Light snow',  
    'Heavy snow', 
    'Patchy heavy snow', 
    'Blowing snow',  
    'Blizzard',
    'Ice pellets', 
    'Patchy light snow', 
    # Rain
    'Patchy rain possible',
    'Light rain shower', 
    'Moderate or heavy rain shower',
    'Moderate rain', 
    'Heavy rain', 
    'Light rain', 
    'Patchy light rain', 
    'Torrential rain shower',
    'Moderate rain at times', 
    'Heavy rain at times', 
    'Moderate or heavy freezing rain',
    # Thunderstorm
    'Thundery outbreaks possible',
    'Moderate or heavy rain with thunder',
    'Patchy light rain with thunder', 
    # Atmosphere
    'Mist',
    'Fog']

In [41]:
weather_condition_codes = [
    # Thunderstorm
    'thunderstorm with light rain',
    'thunderstorm with rain',
    'thunderstorm with heavy rain',
    'light thunderstorm',
    'thunderstorm',
    'heavy thunderstorm',
    'ragged thunderstorm',
    'thunderstorm with light drizzle',
    'thunderstorm with drizzle',
    'thunderstorm with heavy drizzle',
    # Drizzle
    'light intensity drizzle',
    'drizzle',
    'heavy intensity drizzle',
    'light intensity drizzle rain',
    'drizzle rain',
    'heavy intensity drizzle rain',
    'shower rain and drizzle',
    'heavy shower rain and drizzle',
    'shower drizzle',
    # Rain
    'light rain',
    'moderate rain',
    'heavy intensity rain',
    'very heavy rain',
    'extreme rain',
    'freezing rain',
    'light intensity shower rain',
    'shower rain',
    'heavy intensity shower rain',
    'ragged shower rain',
    # Snow
    'light snow',
    'Snow',
    'Heavy snow',
    'Sleet',
    'Light shower sleet',
    'Shower sleet',
    'Light rain and snow',
    'Rain and snow',
    'Light shower snow',
    'Shower snow',
    'Heavy shower snow',
    # Atmosphere
    'mist',
    'Smoke',
    'Haze',
    'sand/ dust whirls',
    'fog',
    'sand',
    'dust',
    'volcanic ash',
    'squalls',
    'tornado',
    # Clear
    'clear sky',
    # Clouds
    'few clouds',
    'scattered clouds',
    'broken clouds',
    'overcast clouds'   
]

In [42]:
weather_icons = [
    "clear sky",
    "few clouds",
    "scattered clouds",
    "broken clouds",
    "shower rain",
    "rain",
    "thunderstorm",
    "snow",
    "mist"
]